# Intro to Recommender Systems Lab

Complete the exercises below to solidify your knowledge and understanding of recommender systems.

For this lab, we are going to be putting together a user similarity based recommender system in a step-by-step fashion. Our data set contains customer grocery purchases, and we will use similar purchase behavior to inform our recommender system. Our recommender system will generate 5 recommendations for each customer based on the purchases they have made.

In [1]:
import pandas as pd
import numpy as np
from scipy.spatial.distance import pdist, squareform

In [3]:
data = pd.read_csv('../data/customer_product_sales.csv')

In [58]:
data["customerName"] = data["FirstName"]+data["LastName"]
data

,CustomerID,FirstName,LastName,SalesID,ProductID,ProductName,Quantity,customerName
0,61288,Rosa,Andersen,134196,229,Bread - Hot Dog Buns,16,RosaAndersen
1,77352,Myron,Murray,6167892,229,Bread - Hot Dog Buns,20,MyronMurray
2,40094,Susan,Stevenson,5970885,229,Bread - Hot Dog Buns,11,SusanStevenson
3,23548,Tricia,Vincent,6426954,229,Bread - Hot Dog Buns,6,TriciaVincent
4,78981,Scott,Burch,819094,229,Bread - Hot Dog Buns,20,ScottBurch
...,...,...,...,...,...,...,...,...
68579,94547,Gerald,Donovan,687884,211,Wine - Prosecco Valdobiaddene,24,GeraldDonovan
68580,57654,Lashonda,Roy,4373415,211,Wine - Prosecco Valdobiaddene,15,LashondaRoy
68581,36751,Glen,Frederick,2405597,211,Wine - Prosecco Valdobiaddene,10,GlenFrederick
68582,43270,Devon,Mc Daniel,4328852,211,Wine - Prosecco Valdobiaddene,11,DevonMc Daniel


In [59]:
len(data["customerName"].unique())

999

In [68]:
"""I have decided to do the lab using customerName in stead of CustomerID, loosing one record"""

'I have decided to do the lab using customerName in stead of CustomerID, loosing one record'

## Step 1: Create a data frame that contains the total quantity of each product purchased by each customer.

You will need to group by CustomerID and ProductName and then sum the Quantity field.

In [60]:
data_1 = data.groupby(["CustomerID","ProductName","customerName"],as_index=False).agg(alvaro_sum=("Quantity","sum"))
data_1

,CustomerID,ProductName,customerName,alvaro_sum
0,33,Apricots - Dried,LindsaySantana,1
1,33,Assorted Desserts,LindsaySantana,1
2,33,Bandage - Flexible Neon,LindsaySantana,1
3,33,"Bar Mix - Pina Colada, 355 Ml",LindsaySantana,1
4,33,"Beans - Kidney, Canned",LindsaySantana,1
...,...,...,...,...
63623,98200,Vol Au Vents,SammyRocha,50
63624,98200,Wasabi Powder,SammyRocha,25
63625,98200,Wine - Fume Blanc Fetzer,SammyRocha,25
63626,98200,Wine - Hardys Bankside Shiraz,SammyRocha,25


## Step 2: Use the `pivot_table` method to create a product by customer matrix.

The rows of the matrix should represent the products, the columns should represent the customers, and the values should be the quantities of each product purchased by each customer. You will also need to replace nulls with zeros, which you can do using the `fillna` method.

In [61]:
data_2 = data_1.pivot_table(index=["ProductName"], columns=["customerName"], values=["alvaro_sum"], aggfunc="sum",fill_value=0)
data_2 = data_2["alvaro_sum"]
data_2_transposed = data_2.T
data_2_transposed

ProductName,Anchovy Paste - 56 G Tube,"Appetizer - Mini Egg Roll, Shrimp",Appetizer - Mushroom Tart,Appetizer - Sausage Rolls,Apricots - Dried,Apricots - Halves,Apricots Fresh,Arizona - Green Tea,Artichokes - Jerusalem,Assorted Desserts,...,"Wine - White, Colubia Cresh","Wine - White, Mosel Gold","Wine - White, Schroder And Schyl",Wine - Wyndham Estate Bin 777,Wonton Wrappers,Yeast Dry - Fermipan,Yoghurt Tubes,"Yogurt - Blueberry, 175 Gr",Yogurt - French Vanilla,Zucchini - Yellow
customerName,,,,,,,,,,,,,,,,,,,,,
AaronEdwards,0,8,0,8,0,0,0,0,0,0,...,0,0,8,8,0,0,0,0,0,8
AaronWashington,2,2,0,0,0,0,0,0,0,0,...,0,2,0,2,0,0,0,0,0,0
AbelHoffman,0,0,0,0,0,0,0,0,0,0,...,21,0,0,21,0,0,0,0,0,0
AbigailCharles,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AdamBarr,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,9,0,0,9
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WilliamWelch,0,0,6,0,0,0,6,6,0,0,...,0,0,0,0,0,0,0,0,0,0
XavierOwen,0,0,0,0,10,0,0,10,0,10,...,0,0,0,0,0,0,0,10,0,10
YeseniaBender,12,6,0,6,6,0,0,0,0,0,...,0,6,0,0,6,0,0,0,0,6


## Step 3: Create a customer similarity matrix using `squareform` and `pdist`. For the distance metric, choose "euclidean."

In [62]:
#pdist()
#squareform()
#1/(1+ dataFrame)
#pd.DataFrame()
pdist_calculation_squared_norm = pd.DataFrame(1/(1+ squareform(pdist(X=data_2_transposed, metric='euclidean'))),
                                             index=data_2_transposed.index,
                                             columns=data_2_transposed.index)
pdist_calculation_squared_norm

customerName,AaronEdwards,AaronWashington,AbelHoffman,AbigailCharles,AdamBarr,AdrianaMoran,AimeeWare,AishaMunoz,AlanaCarroll,AlbertAguirre,...,WarrenRobles,WendyWilkins,WesleyGardner,WesleyHendrix,WesleyOlsen,WilliamWelch,XavierOwen,YeseniaBender,YvonneHale,YvonneMerritt
customerName,,,,,,,,,,,,,,,,,,,,,
AaronEdwards,1.000000,0.015437,0.005453,0.004254,0.009418,0.008547,0.006187,0.015289,0.012533,0.004832,...,0.012076,0.009065,0.005103,0.015326,0.012189,0.012091,0.009354,0.013305,0.006543,0.008768
AaronWashington,0.015437,1.000000,0.005558,0.004375,0.010905,0.009687,0.006398,0.040825,0.018406,0.004842,...,0.017077,0.010180,0.005323,0.042807,0.017555,0.017141,0.010316,0.018145,0.006791,0.009954
AbelHoffman,0.005453,0.005558,1.000000,0.003588,0.005195,0.004964,0.004535,0.005584,0.005491,0.004154,...,0.005386,0.005139,0.004068,0.005559,0.005441,0.005583,0.005278,0.005536,0.004570,0.005070
AbigailCharles,0.004254,0.004375,0.003588,1.000000,0.004189,0.004104,0.003778,0.004361,0.004314,0.003461,...,0.004324,0.004204,0.003473,0.004340,0.004406,0.004306,0.004174,0.004281,0.003863,0.004108
AdamBarr,0.009418,0.010905,0.005195,0.004189,1.000000,0.007699,0.005938,0.011058,0.009924,0.004619,...,0.009604,0.008646,0.005114,0.010948,0.010069,0.009684,0.008264,0.010280,0.006133,0.007773
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
WilliamWelch,0.012091,0.017141,0.005583,0.004306,0.009684,0.009105,0.006156,0.017722,0.014246,0.004857,...,0.012812,0.009425,0.005238,0.017403,0.013295,1.000000,0.009970,0.013426,0.006452,0.008949
XavierOwen,0.009354,0.010316,0.005278,0.004174,0.008264,0.007296,0.005750,0.010298,0.009295,0.004701,...,0.009427,0.007590,0.004884,0.010327,0.009344,0.009970,1.000000,0.009512,0.005855,0.007444
YeseniaBender,0.013305,0.018145,0.005536,0.004281,0.010280,0.008919,0.006314,0.018293,0.014353,0.004835,...,0.012774,0.009882,0.005322,0.018157,0.013210,0.013426,0.009512,1.000000,0.006567,0.009397


## Step 4: Check your results by generating a list of the top 5 most similar customers for a specific CustomerID.

In [81]:
#CUSTOMER CHOOSED = AaronEdwards
data_4_simil = pdist_calculation_squared_norm["AaronEdwards"].sort_values(ascending=False)[1:]
data_4_simil_top5 = data_4_simil.head()
data_4_simil_top5_dic_items = dict(data_4_simil_top5).items()
data_4_simil_top5_dic_items

dict_items([('TraceyWalters', 0.01592200800247441), ('HopeIrwin', 0.01586083871913595), ('LeoCross', 0.015832534577188768), ('TysonShepherd', 0.01575051209013114), ('VirgilSmall', 0.015738615973150457)])

In [101]:
data_4_simil_top5_list = data_4_simil_top5.index.tolist()
data_4_simil_top5_list

['TraceyWalters', 'HopeIrwin', 'LeoCross', 'TysonShepherd', 'VirgilSmall']

## Step 5: From the data frame you created in Step 1, select the records for the list of similar CustomerIDs you obtained in Step 4.

In [102]:
data_5 = data_1[data_1["customerName"].isin(data_4_simil_top5_list)]
data_5

,CustomerID,ProductName,customerName,alvaro_sum
131,264,Apricots - Halves,VirgilSmall,1
132,264,Apricots Fresh,VirgilSmall,1
133,264,Bacardi Breezer - Tropical,VirgilSmall,1
134,264,Bagel - Plain,VirgilSmall,1
135,264,Banana - Leaves,VirgilSmall,1
...,...,...,...,...
6522,9533,"Water, Tap",HopeIrwin,3
6523,9533,Watercress,HopeIrwin,3
6524,9533,Wine - Alsace Gewurztraminer,HopeIrwin,3
6525,9533,Wine - Magnotta - Cab Sauv,HopeIrwin,3


## Step 6: Aggregate those customer purchase records by ProductName, sum the Quantity field, and then rank them in descending order by quantity.

This will give you the total number of each product purchased by the 5 most similar customers to the customer you selected in order from most purchased to least.

In [107]:
data_6 = data_5.groupby(["ProductName"],as_index=False).agg(alvaro_sum=("alvaro_sum","sum")).sort_values(by="alvaro_sum", ascending=False).reset_index(drop=True)
data_6

,ProductName,alvaro_sum
0,Tomatoes Tear Drop,10
1,Appetizer - Sausage Rolls,9
2,Bacardi Breezer - Tropical,9
3,"Pepsi - Diet, 355 Ml",8
4,Berry Brulee,8
...,...,...
220,Smirnoff Green Apple Twist,1
221,Skirt - 29 Foot,1
222,General Purpose Trigger,1
223,Scampi Tail,1


## Step 7: Filter the list for products that the chosen customer has not yet purchased and then recommend the top 5 products with the highest quantities that are left.

- Merge the ranked products data frame with the customer product matrix on the ProductName field.
- Filter for records where the chosen customer has not purchased the product.
- Show the top 5 results.

In [129]:
data_AaronEdwards = data_1[data_1["customerName"].isin(["AaronEdwards"])]
data_AaronEdwards_products = data_AaronEdwards["ProductName"].unique().tolist()
#data_AaronEdwards_products
data_7 = data_6.drop(data_6[data_6["ProductName"].isin(data_AaronEdwards_products)].index).sort_values(by="alvaro_sum", ascending=False).reset_index(drop=True)
data_7.head()

,ProductName,alvaro_sum
0,"Pepsi - Diet, 355 Ml",8
1,Berry Brulee,8
2,Table Cloth 120 Round White,7
3,Mangoes,7
4,Chips Potato Salt Vinegar 43g,7


## Step 8: Now that we have generated product recommendations for a single user, put the pieces together and iterate over a list of all CustomerIDs.

- Create an empty dictionary that will hold the recommendations for all customers.
- Create a list of unique CustomerIDs to iterate over.
- Iterate over the customer list performing steps 4 through 7 for each and appending the results of each iteration to the dictionary you created.

In [166]:
step_8_dic_from_list = {}
step_8_dic_from_dict = {}
step_8_dic_from_df = {}

step_8_customer_name_list = data["customerName"].unique().tolist()

In [167]:
#RECOMENDATIONS LIST
for name in step_8_customer_name_list:
    data_8_step4_simil_list = pdist_calculation_squared_norm[name].sort_values(ascending=False)[1:6].index.tolist()
    data_8_step5_similar_records = data_1[data_1["customerName"].isin(data_8_step4_simil_list)]
    data_8_step6_aggregation = data_8_step5_similar_records.groupby(["ProductName"],as_index=False).agg(alvaro_sum=("alvaro_sum","sum")).sort_values(by="alvaro_sum", ascending=False).reset_index(drop=True)
    data_8_step7_user_product_consumed_df = data_1[data_1["customerName"].isin([name])]
    data_8_step7_user_product_consumed_products = data_8_step7_user_product_consumed_df["ProductName"].unique().tolist()
    data_8_step7_top5 = data_8_step6_aggregation.drop(data_8_step6_aggregation[data_8_step6_aggregation["ProductName"].isin(data_8_step7_user_product_consumed_products)].index).sort_values(by="alvaro_sum", ascending=False).reset_index(drop=True).head()
    
    step_8_dic_from_list[name] = data_8_step7_top5["ProductName"].tolist()
step_8_dic_from_list

{'RosaAndersen': ['Jagermeister',
  'Chicken - Soup Base',
  'Lime Cordial - Roses',
  'Macaroons - Two Bite Choc',
  'Flavouring - Orange'],
 'MyronMurray': ['Wine - Valpolicella Masi',
  'Tahini Paste',
  'Tuna - Salad Premix',
  'Chicken - Soup Base',
  'Isomalt'],
 'SusanStevenson': ['Bread - Italian Corn Meal Poly',
  'Puree - Mocha',
  'Tuna - Salad Premix',
  'Sherry - Dry',
  'Beer - Sleemans Cream Ale'],
 'TriciaVincent': ['Squid - Tubes / Tenticles 10/20',
  'Wanton Wrap',
  'Puree - Mocha',
  'Pernod',
  'Sprouts - Baby Pea Tendrils'],
 'ScottBurch': ['Vanilla Beans',
  'Lettuce - Frisee',
  'Yogurt - Blueberry, 175 Gr',
  'Cinnamon Buns Sticky',
  'Pop Shoppe Cream Soda'],
 'HopeBowers': ['Cocoa Butter',
  'Cheese - Boursin, Garlic / Herbs',
  'Garlic - Peeled',
  'Cheese - Mix',
  'Sauce - Hollandaise'],
 'NicolasHolmes': ['Juice - Lime',
  'Pomello',
  'Tomatoes Tear Drop',
  'Skirt - 29 Foot',
  'Wine - Blue Nun Qualitatswein'],
 'JudithMiranda': ['Bread - French Baquett

In [168]:
#RECOMENDATIONS DICT
for name in step_8_customer_name_list:
    data_8_step4_simil_list = pdist_calculation_squared_norm[name].sort_values(ascending=False)[1:6].index.tolist()
    data_8_step5_similar_records = data_1[data_1["customerName"].isin(data_8_step4_simil_list)]
    data_8_step6_aggregation = data_8_step5_similar_records.groupby(["ProductName"],as_index=False).agg(alvaro_sum=("alvaro_sum","sum")).sort_values(by="alvaro_sum", ascending=False).reset_index(drop=True)
    data_8_step7_user_product_consumed_df = data_1[data_1["customerName"].isin([name])]
    data_8_step7_user_product_consumed_products = data_8_step7_user_product_consumed_df["ProductName"].unique().tolist()
    data_8_step7_top5 = data_8_step6_aggregation.drop(data_8_step6_aggregation[data_8_step6_aggregation["ProductName"].isin(data_8_step7_user_product_consumed_products)].index).sort_values(by="alvaro_sum", ascending=False).reset_index(drop=True).head()
    
    step_8_dic_from_dict[name] = data_8_step7_top5.to_dict()
step_8_dic_from_dict

{'RosaAndersen': {'ProductName': {0: 'Jagermeister',
   1: 'Chicken - Soup Base',
   2: 'Lime Cordial - Roses',
   3: 'Macaroons - Two Bite Choc',
   4: 'Flavouring - Orange'},
  'alvaro_sum': {0: 12, 1: 12, 2: 10, 3: 9, 4: 8}},
 'MyronMurray': {'ProductName': {0: 'Wine - Valpolicella Masi',
   1: 'Tahini Paste',
   2: 'Tuna - Salad Premix',
   3: 'Chicken - Soup Base',
   4: 'Isomalt'},
  'alvaro_sum': {0: 18, 1: 18, 2: 17, 3: 16, 4: 15}},
 'SusanStevenson': {'ProductName': {0: 'Bread - Italian Corn Meal Poly',
   1: 'Puree - Mocha',
   2: 'Tuna - Salad Premix',
   3: 'Sherry - Dry',
   4: 'Beer - Sleemans Cream Ale'},
  'alvaro_sum': {0: 12, 1: 10, 2: 8, 3: 8, 4: 8}},
 'TriciaVincent': {'ProductName': {0: 'Squid - Tubes / Tenticles 10/20',
   1: 'Wanton Wrap',
   2: 'Puree - Mocha',
   3: 'Pernod',
   4: 'Sprouts - Baby Pea Tendrils'},
  'alvaro_sum': {0: 5, 1: 4, 2: 4, 3: 4, 4: 4}},
 'ScottBurch': {'ProductName': {0: 'Vanilla Beans',
   1: 'Lettuce - Frisee',
   2: 'Yogurt - Blueber

In [169]:
#RECOMENDATIONS DF
for name in step_8_customer_name_list:
    data_8_step4_simil_list = pdist_calculation_squared_norm[name].sort_values(ascending=False)[1:6].index.tolist()
    data_8_step5_similar_records = data_1[data_1["customerName"].isin(data_8_step4_simil_list)]
    data_8_step6_aggregation = data_8_step5_similar_records.groupby(["ProductName"],as_index=False).agg(alvaro_sum=("alvaro_sum","sum")).sort_values(by="alvaro_sum", ascending=False).reset_index(drop=True)
    data_8_step7_user_product_consumed_df = data_1[data_1["customerName"].isin([name])]
    data_8_step7_user_product_consumed_products = data_8_step7_user_product_consumed_df["ProductName"].unique().tolist()
    data_8_step7_top5 = data_8_step6_aggregation.drop(data_8_step6_aggregation[data_8_step6_aggregation["ProductName"].isin(data_8_step7_user_product_consumed_products)].index).sort_values(by="alvaro_sum", ascending=False).reset_index(drop=True).head()
    
    step_8_dic_from_df[name] = data_8_step7_top5
step_8_dic_from_df

{'RosaAndersen':                  ProductName  alvaro_sum
 0               Jagermeister          12
 1        Chicken - Soup Base          12
 2       Lime Cordial - Roses          10
 3  Macaroons - Two Bite Choc           9
 4        Flavouring - Orange           8,
 'MyronMurray':                 ProductName  alvaro_sum
 0  Wine - Valpolicella Masi          18
 1              Tahini Paste          18
 2       Tuna - Salad Premix          17
 3       Chicken - Soup Base          16
 4                   Isomalt          15,
 'SusanStevenson':                       ProductName  alvaro_sum
 0  Bread - Italian Corn Meal Poly          12
 1                   Puree - Mocha          10
 2             Tuna - Salad Premix           8
 3                    Sherry - Dry           8
 4       Beer - Sleemans Cream Ale           8,
 'TriciaVincent':                        ProductName  alvaro_sum
 0  Squid - Tubes / Tenticles 10/20           5
 1                      Wanton Wrap           4
 2     

##  Step 9: Store the results in a Pandas data frame. The data frame should a column for Customer ID and then a column for each of the 5 product recommendations for each customer.

In [176]:
step_9 = pd.DataFrame({"customerName":step_8_dic_from_list.keys(),"recomendatedProducts":step_8_dic_from_list.values()})
step_9

,customerName,recomendatedProducts
0,RosaAndersen,"[Jagermeister, Chicken - Soup Base, Lime Cordi..."
1,MyronMurray,"[Wine - Valpolicella Masi, Tahini Paste, Tuna ..."
2,SusanStevenson,"[Bread - Italian Corn Meal Poly, Puree - Mocha..."
3,TriciaVincent,"[Squid - Tubes / Tenticles 10/20, Wanton Wrap,..."
4,ScottBurch,"[Vanilla Beans, Lettuce - Frisee, Yogurt - Blu..."
...,...,...
994,DevinBush,"[Pasta - Penne, Rigate, Dry, Browning Caramel ..."
995,BradfordHolt,"[Veal - Inside, Wonton Wrappers, Veal - Inside..."
996,AnnaHart,"[Olives - Kalamata, Wine - White Cab Sauv.on, ..."
997,JavierRasmussen,"[Veal - Brisket, Provimi,bnls, Beef - Ground M..."


## Step 10: Change the distance metric used in Step 3 to something other than euclidean (correlation, cityblock, consine, jaccard, etc.). Regenerate the recommendations for all customers and note the differences.